In [1]:
from pathlib import Path

tous_fichiers_txt =[]
for fichier in Path("txt").rglob("*.txt"):
     tous_fichiers_txt.append(fichier.parent / fichier.name)

# décompte du nombre de fichiers dans la liste
n_fichiers = len(tous_fichiers_txt)
print(n_fichiers)

366


In [2]:
tous_fichiers_txt.sort()
tous_fichiers_txt[0]

PosixPath('txt/0101.txt')

In [3]:
tous_documents = []
for fichier_txt in tous_fichiers_txt:
    with open(fichier_txt) as f:
        fichier_txt_chaine = f.read()
    tous_documents.append(fichier_txt_chaine)

In [4]:
# importer le vectoriseur TfidfVectorizer de Scikit-Learn.  
from sklearn.feature_extraction.text import TfidfVectorizer
 
vectoriseur = TfidfVectorizer(max_df=.65, min_df=1, stop_words=None, use_idf=True, norm=None)
documents_transformes = vectoriseur.fit_transform(tous_documents)

In [5]:
documents_transformes_tableau = documents_transformes.toarray()
# la prochaine ligne de code vérifie que le tableau numpy contient le même nombre
# de documents que notre liste de fichiers
len(documents_transformes_tableau)

366

In [6]:
import pandas as pd

# créer un répertoire de sortie s'il n'existe pas déjà
Path("./tf_idf_resultats").mkdir(parents=True, exist_ok=True)

In [7]:
# construire une liste de noms de fichiers de résultats à partir de 
# la liste de noms de fichiers de données et du répertoire de sortie
fichiers_resultats = [str(fichier_txt).replace(".txt", ".csv").replace("txt/", "tf_idf_resultats/") for fichier_txt in tous_fichiers_txt]

# traiter chacun des documents du tableau documents_transformes_tableau
# en utilisant enumerate() pour conserver la trace de la position courante dans le tableau
for compteur, document in enumerate(documents_transformes_tableau):
    # construire un objet de la classe DataFrame
    tf_idf_tuples = list(zip(vectoriseur.get_feature_names(), document))
    un_document_format_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['terme', 'pointage']).sort_values(by='pointage', ascending=False).reset_index(drop=True)

    # enregistrer les résultats dans un document CSV, en utilisant
    # la variable 'compteur' pour choisir le nom de fichier
    un_document_format_df.to_csv(fichiers_resultats[compteur])